# All filters generator

### Parameters

In [2]:
coefficients = [
                    [ 1,  -3,  63,   4,  -2, 1],
                    [ 1,  -5,  62,   8,  -3, 1],
                    [ 2,  -8,  60,  13,  -4, 1],

                    [ 3, -10,  58,  17,  -5, 1],
                    [ 3, -11,  52,  26,  -8, 2],
                    [ 2,  -9,  47,  31, -10, 3],
                    [ 3, -11,  45,  34, -10, 3],

                    [ 3, -11,  40,  40, -11, 3],
                    [ 3, -10,  34,  45, -11, 3],
                    [ 3, -10,  31,  47,  -9, 2],
                    [ 2,  -8,  26,  52, -11, 3],

                    [ 1,  -5,  17,  58, -10, 3],
                    [ 1,  -4,  13,  60,  -8, 2],
                    [ 1,  -3,   8,  62,  -5, 1],
                    [ 1,  -2,   4,  63,  -3, 1]
                    ]

In [3]:
# N = 16              # Number of pixels by block
FRAC = 15           # Number of fractional pixels for interpolation (eg: 1/4 of resolution)
# MODE = 0            # Select MC(0) or FME(1):
# SEARCH = 2          # [Only for MC] Select Horizontal search(0), Vertical search(1) or Full search(2)
TAPS = 8            # Number of taps in the filter
# I = 8               # Input size of the line 
# S = 1               # Stride=1 means that the filter will deslocate 1 by 1 pixel at the time

PADDING = 2         # How much padding to add in each four sides

IN_NUM = 6          # 1 line of 6 samples (6-tap)
IN_BIT_WIDTH = 8    # Number of bits per sample

OUT_NUM = 15        # There are 15 filters used to generate 1/16 precision

In [4]:
import math
import numpy as np
from matplotlib import pyplot as plt

### Variables calculation

#### Input and outputs

In [5]:
# Input size
in_width = IN_NUM * IN_BIT_WIDTH

# Input text
in_text = "\tinput signed ["+str(in_width-1)+":0] X;"

# Output ports
out_names = []
for i in range(1,OUT_NUM+1):
    if i != OUT_NUM:
        out_names.append('\tY'+str(i)+',')
    else:
        out_names.append('\tY'+str(i))
out_names_concat = '\n'.join(out_names)

# Output port sizes
out_port_sizes = []
for i in range(0,OUT_NUM):
    out_port_sizes.append(18)

# Output port modes
out_port = []
for i in range(1,OUT_NUM+1):
    out_port.append('\toutput signed ['+str(out_port_sizes[i-1])+':0] Y'+str(i)+';')
out_port_concat = '\n'.join(out_port)


#### Wires

In [6]:
# Wires

#### Coefficients sizes

In [51]:
# Pre calculated in Google Sheets [TCC] Affine interpolation of a 4x4
coefs_sizes = [
                [8,	10,	14, 10,	10,	8],
                [8,	11,	14,	11,	10,	8],
                [9,	12,	14,	12,	11,	8],
                [10, 12, 14, 13, 11,8],
                [10, 12, 14, 13, 12,9],
                [9,	12, 14,	13,	12,	10],
                [10, 12, 14, 14, 12, 10],
                [10, 12, 14, 14, 12, 10],
                [10, 12, 14, 14, 12, 10],
                [10, 12, 13, 14, 12, 9],
                [9,	12,	13,	14,	12,	10],
                [8,	11,	13,	14,	12,	10],
                [8,	11,	12,	14,	12,	9],
                [8,	10,	11,	14,	11,	8],
                [8,	10,	10,	14,	10,	8]]



In [52]:
# coefs_sizes = [[0 for x in range(TAPS-2)] for y in range(FRAC)] 
# max_input = -(2**(IN_BIT_WIDTH-1))

# for j in range(0,TAPS-2):
#     for i in range(0,FRAC):
#         coef = coefficients[i][j]
#         print(coef)
#         if (coef<0 and (math.log(abs(coef),2))%1==0):
#             # a = 1
#             coefs_sizes[i][j] = math.ceil(math.log(abs(max_input*coef),2))+1
#         else:
#             coefs_sizes[i][j] = math.ceil(math.log(abs(max_input*coef),2))+2

# # CEILING(LOG(ABS($E$19*F2),2))+1
# coefs_sizes

#### Filter outputs names

In [57]:
filters_out = []
filters_reg = []
for t in range(0,TAPS-2):
    for y in range(1,FRAC+1):
        c = coefficients[y-1][t]
        s = coefs_sizes[y-1][t]
        if y != 15:
            if c > 0:
                filters_out.append('\t\t.Y'+str(y)+'\t(r_t'+str(t)+'_f'+str(y)+'_'+str(c)+');')
                filters_reg.append('\treg signed ['+str(s-1)+':0] '+'\t(r_t'+str(t)+'_f'+str(y)+'_'+str(c)+');')
            else:
                filters_out.append('\t\t.Y'+str(y)+'\t(r_t'+str(t)+'_f'+str(y)+'_'+str(abs(c))+'neg);')
                filters_reg.append('\treg signed ['+str(s-1)+':0] '+'\t(r_t'+str(t)+'_f'+str(y)+'_'+str(abs(c))+'neg);')
        else:
            if c > 0:
                filters_out.append('\t\t.Y'+str(y)+'\t(r_t'+str(t)+'_f'+str(y)+'_'+str(c)+')')
                filters_reg.append('\treg signed ['+str(s-1)+':0] '+'\t(r_t'+str(t)+'_f'+str(y)+'_'+str(c)+');')
            else:
                filters_out.append('\t\t.Y'+str(y)+'\t(r_t'+str(t)+'_f'+str(y)+'_'+str(abs(c))+'neg)')
                filters_reg.append('\treg signed ['+str(s-1)+':0] '+'\t(r_t'+str(t)+'_f'+str(y)+'_'+str(abs(c))+'neg);')

filters_out_concat = '\n'.join(filters_out)
filters_reg_concat = '\n'.join(filters_reg)

print(filters_reg_concat)

	reg signed [7:0] 	(r_t0_f1_1);
	reg signed [7:0] 	(r_t0_f2_1);
	reg signed [8:0] 	(r_t0_f3_2);
	reg signed [9:0] 	(r_t0_f4_3);
	reg signed [9:0] 	(r_t0_f5_3);
	reg signed [8:0] 	(r_t0_f6_2);
	reg signed [9:0] 	(r_t0_f7_3);
	reg signed [9:0] 	(r_t0_f8_3);
	reg signed [9:0] 	(r_t0_f9_3);
	reg signed [9:0] 	(r_t0_f10_3);
	reg signed [8:0] 	(r_t0_f11_2);
	reg signed [7:0] 	(r_t0_f12_1);
	reg signed [7:0] 	(r_t0_f13_1);
	reg signed [7:0] 	(r_t0_f14_1);
	reg signed [7:0] 	(r_t0_f15_1);
	reg signed [9:0] 	(r_t1_f1_3neg);
	reg signed [10:0] 	(r_t1_f2_5neg);
	reg signed [11:0] 	(r_t1_f3_8neg);
	reg signed [11:0] 	(r_t1_f4_10neg);
	reg signed [11:0] 	(r_t1_f5_11neg);
	reg signed [11:0] 	(r_t1_f6_9neg);
	reg signed [11:0] 	(r_t1_f7_11neg);
	reg signed [11:0] 	(r_t1_f8_11neg);
	reg signed [11:0] 	(r_t1_f9_10neg);
	reg signed [11:0] 	(r_t1_f10_10neg);
	reg signed [11:0] 	(r_t1_f11_8neg);
	reg signed [10:0] 	(r_t1_f12_5neg);
	reg signed [10:0] 	(r_t1_f13_4neg);
	reg signed [9:0] 	(r_t1_f14_3neg);
	

#### Summing all outputs

s_f1_t0_t1 = (MSB & MSB & r_t0_f1_1) + r_t1_f1_3neg;            // 8 + 10 = 11 
s_f1_t2_t3 = r_t2_f1_63 + (MSB & MSB & MSB & MSB & r_t3_f1_4);  // 14 + 10 = 15
s_f1_t4_t5 = r_t4_f1_2neg + (MSB & MSB & r_t5_f1_1);            // 10 + 8 = 11

s_f1_0 = s_f1_t0_t1 + s_f1_t2_t3; // 11 + 15 = 16
s_f1_1 = s_f1_0 + s_f1_t4_t5; // 16 + 11 = 17
s_f1 = s_f1_1 << 6; // Rounding term is a division by 64, 17 -> 11 bits

### Final Verilog text

In [ ]:
verilog = '''/*------------------------------------------------------------------------------
 * File: all_filters.v
 * Date generated: 13/02/2023
 * Date modified: 13/02/2023
 * Author: Bruna Suemi Nagai
 * Description: Concatenating all 15 filters
 *------------------------------------------------------------------------------ */

module all_filters (
        X,
    '''+out_names_concat+'''
);


// ------------------------------------------
// Parameters
// ------------------------------------------

    // Bit width of input data 
    parameter integer IN_SIZE = 'd8;
	 
	 
// ------------------------------------------
// Port mode declarations
// ------------------------------------------

'''+in_text+'''
'''+out_port_concat+'''

// ------------------------------------------
// Wires declarations
// ------------------------------------------

//  wire signed [8:0] w2;
//  wire signed [9:0] w2_;

	 
// ------------------------------------------
// Combinational logic
// ------------------------------------------

//  assign w1 = X;
//  assign w4 = w1 << 2;
//  assign w3 = w4 - w1;
//  assign w5 = w1 + w4;
//  assign w2_ = -1 * w2;

  
// ------------------------------------------
// Outputs
// ------------------------------------------

//  assign Y1 = w2_;


endmodule //all_filters

'''
print(verilog)